## keras.Sequentialを使った簡単なdeep modelの構築

### modelの構築

**Sequential**のインスタンスを作成することでモデルを構築することができる。importしたlayerをインスタンスの引数として与えることでモデルに層を追加することができる。

layerのは次のように作成時にまとめて与えてもいいし，

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])


最初にSequentialインスタンスを作成して，**model.add**メソッドによって追加しても良い。

In [2]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

### shapeの指定

入力層にはどのような形のデータが入ってくるかの情報を与えておく必要がある。入力層以外は前層の出力結果から入力の形が類推できるので指定する必要はない。

- 最初の層の引数として **input_shape** を使う。例えばMNISTであれば， `input_shape=(None, 784)` とする。ここで**Noneは何らかの整数を期待する場合に使用する**。`Batch_size`を変える可能性がある時は, `None`にしておくこと推奨。`input_shape`にはタプルを与えることが期待されている。


- FC層などのベクターを入力として期待する層は, `input_shape`の代わりに`input_dim`(ベクターの長さ)を使うことができる。モノクロ画像のように，　(batch, height, row) という形で入力が期待される層は， `input_dim` と `input_height` を引数として渡すことができる。


- `batch_size`引数を渡すことで, バッチ数を明示的に渡すことができる。


基本的には, **input_shape=(None, height, row, channel, ...)** の形で渡せばok

shape指定の例。2つのコードは質的に等価である。

In [3]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

In [4]:
model = Sequential()
model.add(Dense(32, input_dim=784))

### Compile

**compile**メソッドは, 最適化アルゴリズム, 損失関数, 評価関数の3つを指定する。


- **最適化アルゴリズム**: `keras`によって定義されている識別子(`'rmsprop'`, `'adagrad'`など)を渡すか, `keras.Optimizer`クラスのインスタンスを与える。
- **損失関数**: `keras`によって定義されている識別子(`'categorical_crossentropy'`, `'mse'`など)を渡すか, 目的関数を自分で定義して渡す。
- **評価関数**: `keras`によって定義されている識別子(`metrics=['accuracy']`など)を渡すか, 評価関数を自分で定義して渡す。

In [5]:
# マルチクラス分類問題の場合
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 2値分類問題の場合
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 平均二乗誤差を最小化する回帰問題の場合
model.compile(optimizer='rmsprop',
              loss='mse')

# 独自定義の評価関数を定義
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

### トレーニング

kerasはnp.arrayを入力データ及びラベルデータとして受け取って訓練を行う。この時は**fit**メソッドを使う。fitの引数は次の通り。


- **x**: 入力データ。`np.array`の形で渡す。
- **y**: ラベルデータ。`np.array`の形で渡す。
- **batch_size**: 整数スカラー。一度に訓練するサンプル数を指定する。メモリの制約によるが, 大きい方が勾配ノイズが少なくなる。
- **epochs**: 整数スカラー。トレーニングデータ全体を何回なめるかを指定する。
- **verbose**: 訓練中にログを表示するかのオプション。0で表示を切り, 1でプログレスバー表示, 2で1epochごとにログ表示。
- **callbacks**: `keras.callbacks.Callback`のインスタンスをリストで渡すことで, 訓練中のモデルパラメータの保存やTensorboardの出力を得ることができる。`Callback`インスタンスについては, (https://keras.io/ja/callbacks/) を参照。
- **validation_split**: 検証用データとして残しておくデータの割合を指定する。(0, 1)。
- **validation_data**: 検証用データとして使うデータのタプル。こちらが適用される時は `validation_split`は無視される。
- **shuffle**: エポックごとにデータをシャッフルするかをboolで与える。HDF5データを入力として与える場合は, `'batch'`という引数を渡すことで, バッチサイズのチャンク内でシャッフルすることができる。
- **class_weight**: ディクショナリ形式でクラスごとの重みを格納する。訓練中に損失関数をスケーリングするために使うらしいがよくわからない。
- **sample_weight**: 訓練中に使用するサンプルの重み付けを行う。入力データと同じ長さの`np.array`を与える。
- **initial_epoch**: 訓練開始時のepochを指定する。訓練再開時に使用するオプション。

`fit`を使ってモデルを組んだ時の実装例。

In [6]:
# 1つの入力から2クラス分類をするモデルにおいては

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ダミーデータの作成
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# 各イテレーションのバッチサイズを32で学習を行なう
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.7178 - acc: 0.4820     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7068 - acc: 0.5040     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.6989 - acc: 0.5160     - ETA: 0s - loss: 0.6934 - acc: 0.53
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.6949 - acc: 0.5260     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6865 - acc: 0.5480     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6812 - acc: 0.5660     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6776 - acc: 0.5580     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6737 - acc: 0.5700     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6684 - acc: 0.5850     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6623 - acc: 0.6240     


In [9]:
# 1つの入力から10クラスの分類を行なう場合について（カテゴリ分類）

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ダミーデータ作成
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# ラベルデータをカテゴリの1-hotベクトルにエンコードする
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# 各イテレーションのバッチサイズを32で学習を行なう
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.3372 - acc: 0.1200     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 2.3058 - acc: 0.1130     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 2.2965 - acc: 0.1220     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 2.2867 - acc: 0.1340     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 2.2781 - acc: 0.1450     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 2.2719 - acc: 0.1520     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 2.2644 - acc: 0.1610     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 2.2551 - acc: 0.1690     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 2.2469 - acc: 0.1770     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 2.2351 - acc: 0.1740     
